[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/14MKnDfgWExFIcRbsdz65XeN-Nb-goUL6#scrollTo=sYyTIPLsvMWl)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!wget https://raw.githubusercontent.com/adugnag/S1-S2_Transformer/main/helper.py

from helper import s1_preproc, get_s2_sr_cld_col, apply_cld_shdw_mask, add_cld_shdw_mask, NBRaddTimeline,export_test_image, prepTestData, writeTfrecord
import json
import numpy as np
from pprint import pprint

In [ ]:
#parameters
params = {  'START_DATE': '2019-01-01', 
            'STOP_DATE': '2020-12-31',        
            'ORBIT': 'DESCENDING',
            'RELATIVE_ORBIT_NUMBER': 'ANY', 
            'SATELLITE': 'ANY',
            'POLARIZATION': 'VVVH',
            'ROI': ee.Geometry.Polygon([[[-62.407, -18.249],[-62.407, -18.452],[-62.174, -18.452],[-62.174, -18.249]]]),
            'DEM': ee.Image('USGS/SRTMGL1_003'),
            'APPLY_BORDER_NOISE_CORRECTION': True,
            'APPLY_TERRAIN_FLATTENING': False,
            'TERRAIN_FLATTENING_MODEL': 'VOLUME',
            'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
            'APPLY_SPECKLE_FILTERING': True,
            'SPECKLE_FILTER_FRAMEWORK':'MULTI',
            'SPECKLE_FILTER': 'BOXCAR',
            'SPECKLE_FILTER_KERNEL_SIZE': 15,
            'NR_OF_IMAGES':10,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': False,
            'interval':30,
            'increment':'day',
            'BANDS1' :['VV','VH'],
            'BANDS2' : ['NBR'],
            'DATA_BUCKET':'senalerts_dl4',
            'OUTPUT_BUCKET':'senalerts_dl4',
            'FOLDER':'TDF1',
            'MODEL_NAME':'TDF_MHSA_PAR_S1_S2_v2',
            'IMAGE_FILE_PREFIX1' : 'TDF_TEST_PAR_S1_v0_',
            'IMAGE_FILE_PREFIX2' : 'TDF_TEST_PAR_S2_v0_',
            'NUM_S1':25,
            'NUM_S2':25,
            'OUT_IMAGE_NAME':'TDF_TEST_PAR_S1_S2_Transformer_v4_.TFRecord',
            'OUT_ASSET_NAME': 'TDF_PAR_S1_S2_Transformer_v4_'
            }


In [ ]:
MODEL_DIR = 'gs://' + params['DATA_BUCKET'] + '/' + params['FOLDER'] + '/' + params['MODEL_NAME']
#custom_objects = {'TransformerBlock':TransformerBlock,'PositionalEmbedding':PositionalEmbedding}
hypermodel = tf.keras.models.load_model(MODEL_DIR) 
hypermodel.summary()

In [ ]:
#process Sentinel 1 image collection
s1_processed = s1_preproc(params)
s1_processed = s1_processed.select(['VV','VH'])
s1_nr = s1_processed.size().getInfo()
print('Number of images in the collection: ', s1_processed.size().getInfo())

#apply a moving median smoothing over the time-series
startDate = ee.Date(params['START_DATE']);
secondDate = startDate.advance(params['interval'], params['increment']).millis();
increase = secondDate.subtract(startDate.millis());
LIST1 = ee.List.sequence(startDate.millis(), ee.Date('2020-12-31').millis(), increase);

def mov_s1(date):
  return s1_processed.filterDate(ee.Date(date), ee.Date(date).advance(params['interval'], params['increment'])) \
           .median().set('system:time_start',ee.Date(date).millis())

#smoothened collection
collection1 =  ee.ImageCollection.fromImages(LIST1.map(mov_s1));

#convert the collection to a single image
image1 = collection1.toArrayPerBand()

#Select image to display
image_S1 = collection1.first()
print('Number of images in the smoothened S1 collection: ', collection1.size().getInfo())
 

In [ ]:
#Sentinel-2 ts processing

s2_sr_cld_col = get_s2_sr_cld_col(params['ROI'], params['START_DATE'], params['STOP_DATE'])
s2_sr = s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask)
s2_sr = s2_sr.map(NBRaddTimeline).select('NBR')

print('Number of images in the original collection: ', s2_sr.size().getInfo())

#S2 time-series smoothing
def mov_s2(date):
  return s2_sr.filterDate(ee.Date(date), ee.Date(date).advance(params['interval'], params['increment'])) \
           .median().set('system:time_start',ee.Date(date).millis())

#apply time series smoothing
collection2 =  ee.ImageCollection.fromImages(LIST1.map(mov_s2));

image2 = collection2.toArrayPerBand()
s2_nr = s2_sr.size().getInfo()
print('Number of images in the smoothened S2 collection: ', collection2.size().getInfo())

In [ ]:
#feature maps for exporting the time series data
fmap1 = dict(zip(list(params['BANDS1']), np.repeat(params['NUM_S1'],len(params['BANDS1'])).tolist()))
fmap2 = dict(zip(list(params['BANDS2']), np.repeat(params['NUM_S2'],len(params['BANDS2'])).tolist()))


In [ ]:
#export the test images

export_test_image(fmap1,image1,params['IMAGE_FILE_PREFIX1'])
export_test_image(fmap2,image2,params['IMAGE_FILE_PREFIX2'])

In [ ]:
#@title Utility functions (RUN)
def listFiles(IMAGE_FILE_PREFIX, OUTPUT_BUCKET):
  files_list1 = !gsutil ls 'gs://'{OUTPUT_BUCKET}
  # Get only the files generated by the image export.
  exported_files_list1 = [s for s in files_list1 if IMAGE_FILE_PREFIX in s]

  # Get the list of image files and the JSON mixer file.
  image_files_list1 = []
  json_file1 = None
  for f in exported_files_list1:
    if f.endswith('.tfrecord.gz'):
      image_files_list1.append(f)
    elif f.endswith('.json'):
      json_file1 = f

  # Make sure the files are in the right order.
  image_files_list1.sort()
  return image_files_list1, json_file1

def getMixer(json_file):
  json_text = !gsutil cat {json_file}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(json_text.nlstr)

  return mixer

In [ ]:

image_files_list1, json_file1 = listFiles(params['IMAGE_FILE_PREFIX1'], params['OUTPUT_BUCKET'])
image_files_list2, json_file2 = listFiles(params['IMAGE_FILE_PREFIX2'], params['OUTPUT_BUCKET'])

pprint(image_files_list1)
print(json_file1)

pprint(image_files_list2)
print(json_file2)

In [ ]:
#get the mixer
mixer1 = getMixer(json_file1)
mixer2 = getMixer(json_file2)

print(mixer1)
print(mixer2)

In [ ]:
#Preparte the test Sentinel-1 and Sentinel-2 dataset in  tf.data.Dataset format
image_dataset1 = prepTestData(mixer1,params['BANDS1'],image_files_list1, params['NUM_S1'])
image_dataset2 = prepTestData(mixer2,params['BANDS2'],image_files_list2, params['NUM_S2'])

In [ ]:
def generator():
  for s1, s2 in zip(image_dataset1, image_dataset2):
    yield {"input_5": s1, "input_6": s2}

#Combined Sentinel-1 and 2 datasets
dataset = tf.data.Dataset.from_generator(generator, output_types=({"input_5": tf.float32, "input_6": tf.float32}))

#check both data branches
pprint(iter(dataset).next())

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = hypermodel.predict(dataset, steps=mixer2['totalPatches'], verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

print(predictions.shape)

In [ ]:
OUTPUT_IMAGE_FILE = 'gs://' + params['OUTPUT_BUCKET'] + '/' + params['FOLDER'] + '/' + params['OUT_IMAGE_NAME']
OUTPUT_ASSET_ID ='users/adugnagirma' + '/' + params['OUT_ASSET_NAME']

writeTfrecord(OUTPUT_IMAGE_FILE,OUTPUT_ASSET_ID, predictions, mixer1)

#Upload to GEE
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file1}

In [ ]:
#Upload to GEE
#OUTPUT_ASSET_ID = 'users/adugnagirma' + '/TDF_PAR_S1_S2_Transformer_v2_'
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file1}